In [2]:
import numpy as np
import os
from tqdm import tqdm
import cv2
import glob
import datetime
from skimage.io import imread
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%load_ext tensorboard

import skimage.io as io
import skimage.transform as trans
from skimage import img_as_ubyte
from keras import backend as K


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
from tensorflow.keras import Model, Input
from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU
from keras import backend as K

In [3]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [4]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [5]:
from segmentation_models.utils import set_trainable

In [6]:
# os.chdir('/content/drive/MyDrive/Dermatophyte/Segmentation/Data')
#seed = 20
#np.random.seed = seed

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

TRAIN_PATH = 'C:/Users/hanus/Desktop/Manipal/semester_2/CNN/project/segmentation/data/neutrophils/train/'
TEST_PATH = 'C:/Users/hanus/Desktop/Manipal/semester_2/CNN/project/segmentation/data/neutrophils/test/'

train_ids = os.listdir(TRAIN_PATH + 'image/')
test_ids = os.listdir(TEST_PATH + 'image/')

In [7]:
# OneDrive_1_5-16-2021
path_1 = "C:/Users/hanus/Desktop/Manipal/semester_2/CNN/project/unet/unet/dup_data/neutrophils/OneDrive_1_5-16-2021/train/label/"
emp_img_list = []
os.chdir(path_1)

for filename in os.listdir(path_1):
    img = imread(filename)
    if np.sum(img) == 0:
        emp_img_list.append(filename)
print("Number of images without ROI: ", len(emp_img_list))

# non_emp_img_list = []
# for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
#     if id_ not in emp_img_list:
#         non_emp_img_list.append(id_)

os.chdir('C:/Users/hanus/Desktop/Manipal/semester_2/CNN/project/unet/unet/data')
print('Resizing training images and masks')
X_train = []
Y_train = []
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    if id_ not in emp_img_list:
        img = imread(TRAIN_PATH + '/image/' + id_)
        img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)
        norm_img = np.zeros((800,800))
        img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
        X_train.append(img)
        mask = imread(TRAIN_PATH + '/label/' + id_ )
        mask = cv2.resize(mask, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)
        Y_train.append((mask[:,:,0] / 255).reshape(256, 256, 1))
        
print('Resizing test images and masks')
X_test = []
Y_test = []
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    if id_ not in emp_img_list:
        img1 = imread(TEST_PATH + '/image/' + id_ )
        img1 = cv2.resize(img1, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)
        X_test.append(img1)
        mask1 = imread(TEST_PATH  + '/label/' + id_ )
        mask1 = cv2.resize(mask1, (IMG_HEIGHT, IMG_WIDTH), interpolation=cv2.INTER_NEAREST)
        Y_test.append((mask1[:,:,0] / 255).reshape(256, 256, 1))
        
X_train = np.array(X_train)/255
Y_train = np.array(Y_train)
X_test = np.array(X_test)/255
Y_test = np.array(Y_test)

  4%|██████▎                                                                                                                                                                    | 14/376 [00:00<00:02, 138.60it/s]

Number of images without ROI:  107
Resizing training images and masks


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 122.21it/s]


Resizing test images and masks


In [8]:
print("Shapes :\n======\nX_train = {}\nY_train = {}\nX_test  = {}\nY_test  = {}"
      .format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

Shapes :
X_train = (269, 256, 256, 3)
Y_train = (269, 256, 256, 1)
X_test  = (11, 256, 256, 3)
Y_test  = (11, 256, 256, 1)


In [13]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(X_train,  Y_train, test_size = 0.2, random_state = 0)

# preprocess input
X_train = preprocess_input(x_train)
X_val = preprocess_input(x_val)



#logdir
log_dir = "logsfreez/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpointer = tf.keras.callbacks.ModelCheckpoint('freez.h5', verbose=1, save_best_only=True)
#earlystopper = tf.keras.callbacks.EarlyStopping(patience = 10, monitor='val_loss')
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks = [
              checkpointer,
              tensorboard
            ]

In [14]:
# define model
model = sm.Unet(BACKBONE,encoder_weights='imagenet',classes=1,activation='sigmoid', encoder_freeze = True, decoder_block_type='transpose')
LR = 0.0001
optim =tf.keras.optimizers.Adam(LR)
model.compile(
    optim,
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],

)
#print(model.summary())
# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator


results=model.fit(X_train,y_train,
                  batch_size=16,
                  epochs=10,
                  validation_data=(X_val, y_val),
                  verbose = 1,
                  callbacks=callbacks)



C:\Users\hanus\anaconda3\envs\CNN\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/10
14/14 [==============================] - 34s 783ms/step - loss: 1.4354 - iou_score: 0.0553 - val_loss: 1.6045 - val_iou_score: 0.0679

Epoch 00001: val_loss improved from inf to 1.60446, saving model to freez.h5
Epoch 2/10
14/14 [==============================] - 3s 220ms/step - loss: 1.3504 - iou_score: 0.0652 - val_loss: 1.6177 - val_iou_score: 0.0680

Epoch 00002: val_loss did not improve from 1.60446
Epoch 3/10
14/14 [==============================] - 3s 219ms/step - loss: 1.2816 - iou_score: 0.0829 - val_loss: 1.6112 - val_iou_score: 0.0680

Epoch 00003: val_loss did not improve from 1.60446
Epoch 4/10
14/14 [==============================] - 3s 219ms/step - loss: 1.2217 - iou_score: 0.1045 - val_loss: 1.6131 - val_iou_score: 0.0680

Epoch 00004: val_loss did not improve from 1.60446
Epoch 5/10
14/14 [==============================] - 3s 220ms/step - loss: 1.1756 - iou_score: 0.1248 - val_loss: 1.6221 - val_iou_score: 0.0681

Epoch 00005: val_loss did not improve from 1

In [15]:
from keras.models import load_model
model.load_weights('freez.h5') 

In [16]:
# for layer in model.layers:
#     layer.trainable = True
# model.compile(
#     model.optimizer,
#     loss=sm.losses.bce_jaccard_loss,            #I re-assing the original loss function
#     metrics=[sm.metrics.iou_score],  #I re-assing the original metric function
# #   loss=model.loss,
# #   metrics=model.metrics,
#     loss_weights=model.loss_weights,
#     sample_weight_mode=model.sample_weight_mode,
# #    weighted_metrics=model.weighted_metrics,
#         )

In [17]:
# unfreeze encoder
set_trainable(model)

# results=model.fit(X_train,y_train,
#                   batch_size=16,
#                   epochs=10,
#                   validation_data=(X_val, y_val),
#                   verbose = 1,
#                   callbacks=callbacks)

AttributeError: 'Functional' object has no attribute 'loss_weights'

In [ ]:
model.compile(
    optim,
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)
#print(model.summary())
# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator


results=model.fit(X_train,y_train,
                  batch_size=16,
                  epochs=100,
                  validation_data=(X_val, y_val),
                  verbose = 1,
                  callbacks=callbacks)

In [ ]:
%tensorboard --logdir logsfreez/fit

In [ ]:
from keras.models import load_model
model.load_weights('freez.h5') 

In [ ]:
preds_test = model.predict(X_test, verbose=1)

preds_test_t = (preds_test > 0.3).astype(np.uint8)

10/10 [==============================] - 9s 446ms/step


In [ ]:
preds_test_t = preds_test_t.astype(float)
preds_test=preds_test.astype(float)
for i in range(len(X_test)):
    # print(i)
    plt.figure( figsize=(20,3) )
    plt.subplot(141)
    plt.imshow(X_test[i])
    plt.title('Image')
    plt.subplot(142)
    plt.imshow(Y_test[i])
    plt.title('Mask')
    plt.subplot(143)
    plt.imshow(preds_test_t[i,:,:,0])
    plt.title('Prediction')
    # plt.title('Prediction. Jaccard coeff: ' + str(np.around(jacard_coef(Y_test[i], preds_test_t[i,:,:,0]), 3)))
    # plt.subplot(144)
    # plt.imshow(preds_test[i,:,:,0])
    # plt.title('no threshold mask. Jaccard coeff: ' + str(np.around(jacard_coef(Y_test[i], preds_test[i,:,:,0]), 3)))
    plt.show()

In [ ]:
	# evaluate model
_, acc = model.evaluate(X_test, Y_test)
print("Accuracy of Jacard Model is = ", (acc * 100.0), "%")

10/10 [==============================] - 3s 241ms/step - loss: 9.7522 - iou_score: 0.0567
Accuracy of Jacard Model is =  5.670937895774841 %
